In [1]:
import pandas as pd
import os
import pickle as pkl
from pprint import pprint
from plotly import express as px
from lightgbm import LGBMRegressor

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.model.utils import qwk_obj

In [4]:
df = pd.DataFrame()
names = None

for fold in range(config.lgbm_n_folds):
    model: LGBMRegressor = pkl.load(open(f"output/LGBM/{fold}.pkl", "rb"))["model"]
    importances = model.feature_importances_

    if not names:
        names = model.feature_name_
        df = pd.concat([df, pd.DataFrame({"name": names})], axis=1)

    temp = pd.DataFrame({f"LGBM_{fold}": importances})
    df = pd.concat([df, temp], axis=1)

In [5]:
df.head(10)

,name,LGBM_0,LGBM_1,LGBM_2,LGBM_3,LGBM_4,LGBM_5,LGBM_6,LGBM_7,LGBM_8,...,LGBM_15,LGBM_16,LGBM_17,LGBM_18,LGBM_19,LGBM_20,LGBM_21,LGBM_22,LGBM_23,LGBM_24
0,deberta_m0_c0,184,128,115,153,115,134,77,99,104,...,126,81,87,137,130,109,65,99,138,123
1,deberta_m0_c1,150,116,101,138,126,115,60,96,95,...,106,77,92,124,138,109,66,109,112,120
2,deberta_m0_c2,127,112,116,112,127,128,105,104,118,...,112,114,111,127,140,105,92,128,120,125
3,deberta_m0_c3,118,90,69,94,72,85,35,66,69,...,77,43,50,82,84,66,42,64,75,85
4,deberta_m0_c4,108,79,65,76,87,87,39,59,54,...,79,56,56,92,78,70,32,54,63,72
5,deberta_m0_c5,66,38,48,55,55,57,29,34,42,...,38,38,36,65,52,41,28,34,45,55
6,deberta_m1_c0,87,64,64,89,68,68,30,46,61,...,72,43,37,77,69,42,29,50,60,93
7,deberta_m1_c1,83,52,64,69,68,73,40,57,52,...,52,57,59,64,71,60,49,57,70,51
8,deberta_m1_c2,116,95,103,124,110,122,72,81,99,...,122,83,98,132,123,110,79,91,113,111
9,deberta_m1_c3,67,47,39,59,56,63,37,50,40,...,49,34,37,70,50,45,39,49,44,53


In [6]:
df["importance"] = df.loc[:, "LGBM_0":].median(axis=1)
df.sort_values(by="importance", ascending=False, inplace=True)
df = df[["name", "importance"]]

## Overall

Top 10 most important features

In [7]:
df.head(10)

,name,importance
18,deberta_m3_c0,121.0
20,deberta_m3_c2,120.0
0,deberta_m0_c0,114.0
2,deberta_m0_c2,114.0
1,deberta_m0_c1,109.0
12,deberta_m2_c0,108.0
8,deberta_m1_c2,105.0
14,deberta_m2_c2,92.0
86,sentence_word_count_sum,82.0
50,paragraph_word_count_sum,82.0


Top 10 least important features

In [8]:
df.tail(10)

,name,importance
160,tfidf_51,0.0
94,sentence_word_count_len_goe_1,0.0
158,tfidf_49,0.0
154,tfidf_45,0.0
151,tfidf_42,0.0
146,tfidf_37,0.0
145,tfidf_36,0.0
144,tfidf_35,0.0
143,tfidf_34,0.0
159,tfidf_50,0.0


## Broad Groups

In [9]:
def get_broad_group(name):
    group = name.split("_")[0]

    if group == "tfidf" and name.split("_")[1] == "count":
        return "tfidf_count"

    return group

In [10]:
df["broad_group"] = df["name"].map(get_broad_group)

In [11]:
px.box(
    df,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

From an high level, it looks like
* `score` probabilities from DeBERTa model has he highest importance.
* `Paragraph` and `sentence` features also seem to be quite important as well.
* `word` and `tfidf_count` seems to be the least important.

In [12]:
def get_count_type(name):
    return "_".join(name.split("_")[1:3])

In [13]:
df["count_type"] = df["name"].map(get_count_type)

## Broad Operations

In [14]:
def get_operation_type(name):
    last_word = name.split("_")[-1]
    if last_word in [
        "sum",
        "max",
        "mean",
        "min",
        "q1",
        "q3",
    ]:
        return last_word

    if name.find("len_goe") != -1:
        return "greater_than"
    elif name.find("len_l") != -1:
        return "less_than"
    return "None"

In [15]:
df["operation_type"] = df["name"].map(get_operation_type)

In [16]:
px.box(
    df,
    x="importance",
    color="operation_type",
    range_x=(df.importance.min(), df.importance.max()),
)

Broadly, `min`, `kurtosis`, `q1`, `first` are worthless, regardless of broad group.

### Greater Than

In [17]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [18]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [19]:
df[(df.operation_type == "greater_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
69,paragraph_sentence_count_len_goe_16,27.0,paragraph,sentence_count,greater_than
77,paragraph_error_count_len_goe_16,17.0,paragraph,error_count,greater_than


**Verdict**

1. For paragraphs
   1. Count only sentence and error. Upper limit be 15.
2. For sentences
   1. Count only error and word. Upper limit be 20.

### Less Than

In [20]:
px.box(
    df[df.operation_type == "less_than"],
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

In [21]:
px.box(
    df[df.operation_type == "greater_than"],
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [22]:
df[(df.operation_type == "less_than") & (df.importance >= 10)]

,name,importance,broad_group,count_type,operation_type
68,paragraph_sentence_count_len_l_16,46.0,paragraph,sentence_count,less_than
76,paragraph_error_count_len_l_16,19.0,paragraph,error_count,less_than
99,sentence_word_count_len_l_16,10.0,sentence,word_count,less_than


**Verdict**

1. For paragraphs: Count only sentence and error. Upper limit be 20.
2. For sentences: Count only error and word. Upper limit be 20.

## Paragraph Features

In [23]:
paragraph_features = df.loc[df.broad_group == "paragraph"]
print("Shape of paragraph features:", paragraph_features.shape)

Shape of paragraph features: (36, 5)


In [24]:
px.box(
    paragraph_features,
    x="importance",
    color="count_type",
    range_x=(df.importance.min(), df.importance.max()),
)

In [25]:
paragraph_features[
    (paragraph_features.count_type == "char_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
47,paragraph_char_count_sum,38.0,paragraph,char_count,sum
46,paragraph_char_count_max,35.0,paragraph,char_count,max
45,paragraph_char_count_mean,24.0,paragraph,char_count,mean
57,paragraph_char_count_q3,20.0,paragraph,char_count,q3
56,paragraph_char_count_q1,7.0,paragraph,char_count,q1


In [26]:
paragraph_features[
    (paragraph_features.count_type == "word_count")
    & (paragraph_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
50,paragraph_word_count_sum,82.0,paragraph,word_count,sum
48,paragraph_word_count_mean,25.0,paragraph,word_count,mean
49,paragraph_word_count_max,25.0,paragraph,word_count,max
59,paragraph_word_count_q3,22.0,paragraph,word_count,q3
58,paragraph_word_count_q1,9.0,paragraph,word_count,q1


## TFIDF and Count

In [27]:
tfidf_features = df.loc[df.broad_group == "tfidf"]
print("Shape of tfidf features:", tfidf_features.shape)

Shape of tfidf features: (100, 5)


In [28]:
px.box(
    tfidf_features,
    x="importance",
    color="broad_group",
    range_x=(df.importance.min(), df.importance.max()),
)

TFIDF count features are worthless.

In [29]:
tfidf_features[
    (tfidf_features.broad_group == "tfidf") & (tfidf_features.importance > 0)
]

,name,importance,broad_group,count_type,operation_type
135,tfidf_26,40.0,tfidf,26,None
109,tfidf_0,30.0,tfidf,0,None
110,tfidf_1,18.0,tfidf,1,None
116,tfidf_7,10.0,tfidf,7,None
176,tfidf_67,8.0,tfidf,67,None
208,tfidf_99,3.0,tfidf,99,None
117,tfidf_8,3.0,tfidf,8,None
134,tfidf_25,3.0,tfidf,25,None
150,tfidf_41,3.0,tfidf,41,None
167,tfidf_58,2.0,tfidf,58,None


## DeBERTa Features

In [30]:
deberta_features = df.loc[df.broad_group.isin(["deberta"])]
print("Shape of deberta features:", deberta_features.shape)

Shape of deberta features: (42, 5)


In [31]:
def get_model_num(name):
    return int(name.split("_")[1][1])


deberta_features["model"] = deberta_features.name.map(get_model_num)

/tmp/ipykernel_17285/962180890.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
px.box(
    deberta_features,
    x="importance",
    color="model",
    range_x=(df.importance.min(), df.importance.max()),
)

### Textstat

In [33]:
textstat_features = df.loc[df.broad_group.isin(["textstat"])]
print("Shape of deberta features:", textstat_features.shape)

Shape of deberta features: (16, 5)


In [34]:
textstat_features["function"] = textstat_features.name.map(lambda x: x[9:])

/tmp/ipykernel_17285/3565201953.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
textstat_features[textstat_features.importance > 5]

,name,importance,broad_group,count_type,operation_type,function
215,textstat_difficult_words,14.0,textstat,difficult_words,None,difficult_words
